In [1]:
import torch as t
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [2]:
class traindata(Dataset):
    def __init__(self):
        xy = np.loadtxt('train.csv', dtype='int', delimiter=',')
        '''
        li = [0, 5, 8, 10, 22]
        for i in li:
            xy[:, i] = xy[:, i] / 100
        '''
        self.y_data = t.from_numpy(xy[:, 11])
        xy = np.delete(xy, 11, axis=1)
        self.x_data = t.from_numpy(xy)
        self.len = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fn1 = nn.Linear(43, 5)
        self.fn2 = nn.Linear(5, 5)
        #self.fn3 = nn.Linear(5, 5)
        #self.fn4 = nn.Linear(5, 5)
        self.fn5 = nn.Linear(5, 1)
        
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        x = t.sigmoid(self.fn1(x))
        x = t.sigmoid(self.fn2(x))
        #x = t.sigmoid(self.fn3(x))
        #x = t.sigmoid(self.fn4(x))
        x = self.fn5(x)
        return x

In [4]:
net = Net()
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [5]:
mytraindata = traindata()
trainloader = DataLoader(dataset = mytraindata, batch_size = 32, shuffle = True, num_workers = 0)

In [6]:
for epoch in range(6):
    

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        
        inputs, labels = data
        inputs = Variable(inputs).float()
        labels = Variable(labels.float()).view(labels.shape[0], -1)
 
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.data.item()
        if i % 4000 == 3999:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/200))
            running_loss = 0.0
print('Finished')


[1,  4000] loss: 7.555
[1,  8000] loss: 5.188
[1, 12000] loss: 3.835
[1, 16000] loss: 3.276
[1, 20000] loss: 3.141
[2,  4000] loss: 3.154
[2,  8000] loss: 3.127
[2, 12000] loss: 3.135
[2, 16000] loss: 3.159
[2, 20000] loss: 3.131
[3,  4000] loss: 3.152
[3,  8000] loss: 3.137
[3, 12000] loss: 3.160
[3, 16000] loss: 3.132
[3, 20000] loss: 3.135
[4,  4000] loss: 3.158
[4,  8000] loss: 3.113
[4, 12000] loss: 3.163
[4, 16000] loss: 3.118
[4, 20000] loss: 3.140
[5,  4000] loss: 3.129
[5,  8000] loss: 3.163
[5, 12000] loss: 3.140
[5, 16000] loss: 3.120
[5, 20000] loss: 3.146
[6,  4000] loss: 3.133
[6,  8000] loss: 3.143
[6, 12000] loss: 3.135
[6, 16000] loss: 3.143
[6, 20000] loss: 3.141
Finished


In [7]:
class Testdata(Dataset):
    def __init__(self):
        xy = np.loadtxt('test.csv', dtype='int', delimiter=',')
        
        self.x_data = t.from_numpy(xy)
        self.len = xy.shape[0]
        
    def __getitem__(self, index):
        return self.x_data[index]
    
    def __len__(self):
        return self.len

In [8]:
mytestdata = Testdata()
testloader = DataLoader(dataset = mytestdata, batch_size = 1, shuffle = False, num_workers = 0)

In [17]:
pred = []
for i, data in enumerate(testloader, 0):
    x = data
    outputs = net(Variable(x).float())
    pred.append(outputs.data[0])
pred = t.Tensor(pred)
    

In [19]:
idd = t.arange(800000, 1000000).view(-1, 1)
print(idd.shape, pred.shape)
pred = t.cat([idd, pred.view(-1, 1)], dim = 1)
pred.shape

torch.Size([200000, 1]) torch.Size([200000])


torch.Size([200000, 2])

In [20]:
np.savetxt('pred2.csv', pred, fmt = '%.4f', delimiter=',')

In [21]:
pred>0.3

tensor([[False],
        [False],
        [False],
        ...,
        [False],
        [False],
        [False]])